# Setup git cloning


In [1]:
! git clone https://github.com/anushagj/friend-up-your-cash-app-game.git
! pip install prefect==1.0 -U
! pip install parquet

Cloning into 'friend-up-your-cash-app-game'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 76 (delta 39), reused 76 (delta 39), pack-reused 0
Receiving objects: 100% (76/76), 2.28 MiB | 18.51 MiB/s, done.
Resolving deltas: 100% (39/39), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.7/575.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 10.8 MB/s eta 0:00:00
  Created wheel for pendulum: filename=pendulum-2.1.2-cp310-cp310-manylinux_2_35_x86_64.whl size=158444 sha256=b896

# Let's quickly explore the data

In [2]:
# Load the sample data into a dataframe. A dataframe is a data structure that organizes data into a 2-dimensional table of rows and columns.

import pandas as pd
parquet_file_path = '/content/friend-up-your-cash-app-game/Dataset/cash_friends.parquet'
cash_friends = pd.read_parquet(parquet_file_path)
cash_friends.head()

,user_id,account_creation_date,gender,count_num_transactions_last_yr,sum_amount_spent_all_time_usd,current_cash_account_balance_usd,current_bitcoin_account_balance_btc,current_stock_account_balance_usd,cash_card_enabled,direct_deposit_enabled,cash_boost_used,most_interacted_user_index,user_occupation,location,most_used_cash_app_feature,account_age_yr,most_interacted_user_id
0,LyuLjUo0dH,2020-04-01,Male,14,1383.0,714.0,2.27,1432.0,Yes,Yes,Yes,442,Lawyer,Wyoming,Peer to Peer Payment,3,dt8BG7TNjO
1,86lAOsc1Gh,2015-07-19,Male,15,528.0,262.0,3.52,2525.0,Yes,No,Yes,842,Lawyer,Connecticut,Investing,8,3WfkGmY1HF
2,YcI21zkiL1,2019-04-23,Female,16,720.0,753.0,1.65,2686.0,No,Yes,No,4698,Designer,Illinois,Cash Card,4,TSFnHGhvcb
3,10zlKlUH4r,2018-11-29,Male,30,1062.0,736.0,2.35,2751.0,No,No,Yes,467,Doctor,Washington,Cash Card,4,vH6YkrHISj
4,dflMuC8Yz8,2015-10-06,Male,11,199.0,350.0,2.53,1550.0,No,Yes,No,3724,Nurse,Kentucky,Boost,7,zTbIUEjCJJ


# Part 1: GCP and Prefect Setup

# Create a Free Google Account
Create a google account [here](https://https://accounts.google.com/signup/v2/createaccount?continue=https%3A%2F%2Fmyaccount.google.com%3Futm_source%3Daccount-marketing-page%26utm_medium%3Dcreate-account-button&flowName=GlifWebSignIn&flowEntry=SignUp) (if you don’t already have one)

# Create a Free Google Cloud Platform Account

In a new tab go to https://console.cloud.google.com/. Then in the top left, click on Select a **project > new project**

# Create a Free-Tier Prefect Account

1.   In a new tab, go to https://cloud.prefect.io/
2.   Click Sign in with Google option and use the new google account created in the previous step.
3.   Click Next, then click TO THE DASHBOARD

# Create Prefect Task

In [ ]:
import prefect
from prefect import task, Flow
@task
def hello_task():
  logger = prefect.context.get("logger")
  logger.info("Hello world!")

flow = Flow("hello-flow", tasks=[hello_task])
flow.run()

# Setup Prefect Cloud with the Python SDK

In [ ]:
! prefect auth login --key <Your KEY>

# Connecting local prefect to our cloud prefect


Create an API key : https://cloud.prefect.io/user/keys, **save the key**!

In [ ]:
! prefect create project cash_find_friends

In [ ]:
flow.register(project_name="cash_find_friends")

Next we follow the link that was generated and select quick run and we will see our flow run in the cloud !



---



In [ ]:
! prefect agent local start



---



# **Create a Table in Big Query using Prefect**

In [ ]:
import parquet
import os
from google.cloud import bigquery
from prefect import task, Flow, Parameter
import pandas as pd

#TO BE UPDATED BY YOU
PROJECT_ID = "ghc23-394604"
DATASET_NAME = "Friends"
TABLE_NAME = "cash_friends"

#TO BE UPDATED BY YOU
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = <Your JSON FILE>

# Function to create a new table in BigQuery
def create_table(project_id, dataset_name, table_name):
  client = bigquery.Client(project=project_id)

  # Define the schema for your table (change the fields accordingly)
  schema = [
    bigquery.SchemaField("user_id", "STRING"),
    bigquery.SchemaField("account_creation_date", "STRING"),
    bigquery.SchemaField("gender", "STRING"),
    bigquery.SchemaField("count_num_transactions_last_yr", "INTEGER"),
    bigquery.SchemaField("sum_amount_spent_all_time_usd", "FLOAT"),
    bigquery.SchemaField("current_cash_account_balance_usd", "FLOAT"),
    bigquery.SchemaField("current_bitcoin_account_balance_btc", "FLOAT"),
    bigquery.SchemaField("current_stock_account_balance_usd", "FLOAT"),
    bigquery.SchemaField("cash_card_enabled", "STRING"),
    bigquery.SchemaField("direct_deposit_enabled", "STRING"),
    bigquery.SchemaField("cash_boost_used", "STRING"),
    bigquery.SchemaField("most_interacted_user_index", "INTEGER"),
    bigquery.SchemaField("user_occupation", "STRING"),
    bigquery.SchemaField("location", "STRING"),
    bigquery.SchemaField("most_used_cash_app_feature", "STRING"),
    bigquery.SchemaField("account_age_yr","INTEGER"),
    bigquery.SchemaField("most_interacted_user_id","STRING")
  ]

  table_ref = client.dataset(dataset_name).table(table_name)
  table = bigquery.Table(table_ref, schema=schema)

  # Create the table
  table = client.create_table(table)
  print(f"Table {table.project}.{table.dataset_id}.{table.table_id} created.")


# Create the table (only needed if the table doesn't already exist)
create_table(PROJECT_ID, DATASET_NAME, TABLE_NAME)

# **Upload data from the parquet file into BigQuery**

In [ ]:
# Function to upload Parquet data to BigQuery table
def upload_parquet_to_bigquery(parquet_file_path, project_id, dataset_name, table_name):
  df = pd.read_parquet(parquet_file_path)

  df['account_creation_date'] = df['account_creation_date'].dt.strftime('%Y-%m-%d %H:%M:%S')


  # Initialize a BigQuery client
  client = bigquery.Client()


  # Define the job configuration
  job_config = bigquery.LoadJobConfig()
  job_config.source_format = bigquery.SourceFormat.PARQUET
  job_config.autodetect = True  # Automatically detect schema

  # Upload the DataFrame to BigQuery
  table_ref = client.dataset(dataset_name).table(table_name)
  job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

  # Wait for the job to complete
  job.result()

  print(f"Loaded {job.output_rows} rows into {dataset_name}:{table_name}")

# Upload the CSV data to the table
upload_parquet_to_bigquery(parquet_file_path, PROJECT_ID, DATASET_NAME, TABLE_NAME)

# Part 2: Encoding & Embeddings

## Setup

In [4]:
import pandas as pd
from sklearn import preprocessing
from scipy.spatial import distance

In [3]:
cash_friends = df

## Encode Cash Friends Categorical Features

In [5]:
categorical_cols = ["user_occupation", "most_used_cash_app_feature", "gender"]
binary_cols = ["cash_card_enabled", "direct_deposit_enabled", "cash_boost_used", ]


In [6]:
# Encode the categorical columns

# use built in encoder preprocessing.LabelEncoder()
def encode_categorical_columns(cols, cash_friends):
    categorical_encoders = {}
    for col in cols:
        # create new label encoder for this column
        label_encoder = preprocessing.LabelEncoder()
        # Fit label encoder to the column values and return encoded labels.
        encoded_col = label_encoder.fit_transform(cash_friends[col].values.tolist())
        # save encoded column values in new column
        cash_friends[col + "_encoded"] = encoded_col
        # save encoder for this column
        categorical_encoders[col] = label_encoder
    return cash_friends, categorical_encoders

In [7]:
# Encode the binary columns

# use built in encoder preprocessing.LabelBinarizer()
def encode_binary_columns(cols, cash_friends):
    binary_encoders = {}
    # TODO create new label Binarizer for this column
    # TODO fit label encoder to the column values and return encoded labels (using fit_transform)
    # TODO save encoded column in new col in cash_friends
    # TODO save encoder for this column in the binary_encoders dict
    return cash_friends, binary_encoders

In [8]:
# Encode the columns
cash_friends, categorical_encoders = encode_categorical_columns(categorical_cols, cash_friends)
cash_friends, binary_encoders = encode_binary_columns(binary_cols, cash_friends)

## Drop all original columns categorical & binary columns

In [9]:
# Drop non numerical columns for distance calculation
vector_df = cash_friends.drop(columns=['user_id', 'most_interacted_user_id', 'account_creation_date', 'gender', 'cash_card_enabled', 'direct_deposit_enabled', 'cash_boost_used', 'user_occupation', 'location',
       'most_used_cash_app_feature'])

## Compute Vector Distances

In [10]:
# use scipy distance functions
# manhattan : distance.cityblock
# euclidean : distance.euclidean


def manhattan_distance(vector_1, vector_2):
    return distance.cityblock(vector_1, vector_2)

def euclidean_distance(vector_1, vector_2):
    # TODO: COMPLETE FUNCTION TO compute distance
    # and return distance
    return ...


## Lets get the top 3 recommended friends for user 0

In [11]:
# Using row 0 as our target row
target_row = vector_df.iloc[0]

In [12]:
# Compute vector distances
manhatten_distances = vector_df.apply(lambda row: manhattan_distance(target_row, row), axis=1)
euclidian_distances = vector_df.apply(lambda row: euclidean_distance(target_row, row), axis=1)
vector_df["manhattan_distances"] = manhatten_distances
vector_df["euclidian_distances"] = euclidian_distances

### Rank the other users and get the top 3 recommended for each distance metric

In [13]:
euclidian_distances = vector_df["euclidian_distances"]
euc_dict = euclidian_distances.to_dict()
ordered_customers_euc =[(customer, distance) for customer, distance in euc_dict.items()]
ordered_customers_euc.sort(key=lambda elem: elem[1])
ordered_customers_euc[:4]


[(0, 0.0),
 (1772, 206.0826360953295),
 (981, 280.55483314318434),
 (2443, 300.237612733648)]

In [14]:
manhattan_distances = vector_df["manhattan_distances"]
man_dict = manhattan_distances.to_dict()
ordered_customers_man =[(customer, distance) for customer, distance in man_dict.items()]
ordered_customers_man.sort(key=lambda elem: elem[1])
ordered_customers_man[:4]

[(0, 0.0), (1772, 304.23), (1183, 499.1), (3320, 526.31)]

### Compare target user to recommended users

In [17]:
target_user = cash_friends.iloc[0]
target_user

user_id                                          LyuLjUo0dH
account_creation_date                   2020-04-01 00:00:00
gender                                                 Male
count_num_transactions_last_yr                           14
sum_amount_spent_all_time_usd                        1383.0
current_cash_account_balance_usd                      714.0
current_bitcoin_account_balance_btc                    2.27
current_stock_account_balance_usd                    1432.0
cash_card_enabled                                       Yes
direct_deposit_enabled                                  Yes
cash_boost_used                                         Yes
most_interacted_user_index                              442
user_occupation                                      Lawyer
location                                            Wyoming
most_used_cash_app_feature             Peer to Peer Payment
account_age_yr                                            3
most_interacted_user_id                 

In [19]:
# Check recommender user using Euclidean distance

In [22]:
recommender_user_id = ordered_customers_euc[1][0]
recommended_user = cash_friends.iloc[recommender_user_id]
recommended_user

user_id                                         FeKVVsuTml
account_creation_date                  2020-06-16 00:00:00
gender                                              Female
count_num_transactions_last_yr                          16
sum_amount_spent_all_time_usd                       1377.0
current_cash_account_balance_usd                     698.0
current_bitcoin_account_balance_btc                   2.04
current_stock_account_balance_usd                   1618.0
cash_card_enabled                                       No
direct_deposit_enabled                                  No
cash_boost_used                                         No
most_interacted_user_index                             529
user_occupation                               Entrepreneur
location                                        Washington
most_used_cash_app_feature                  Direct Deposit
account_age_yr                                           3
most_interacted_user_id                         aL8IUZbB

In [ ]:
# Check recommender user for Manhanttan distance

In [23]:
recommender_user_id = ordered_customers_man[1][0]
recommended_user = cash_friends.iloc[recommender_user_id]
recommended_user

user_id                                         FeKVVsuTml
account_creation_date                  2020-06-16 00:00:00
gender                                              Female
count_num_transactions_last_yr                          16
sum_amount_spent_all_time_usd                       1377.0
current_cash_account_balance_usd                     698.0
current_bitcoin_account_balance_btc                   2.04
current_stock_account_balance_usd                   1618.0
cash_card_enabled                                       No
direct_deposit_enabled                                  No
cash_boost_used                                         No
most_interacted_user_index                             529
user_occupation                               Entrepreneur
location                                        Washington
most_used_cash_app_feature                  Direct Deposit
account_age_yr                                           3
most_interacted_user_id                         aL8IUZbB

# (BONUS) Part 3: Model Training